In [1]:
import plotly.express as px
import lostructpy.lostruct as ls
import numpy as np
from skbio.stats.ordination import pcoa
from sklearn.manifold import MDS
import pandas as pd
import umap
import hdbscan
import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [2]:
windows, positions = ls.parse_vcf("chr1-filtered.vcf.gz", "chr1", 95)

In [3]:
samples = ls.get_samples("chr1-filtered.vcf.gz")

In [29]:
# Code originally used to generate and save random weights. Saved and static now to feed into R and compare with lostruct R
#weights = np.random.random_sample(len(samples))
#np.savetxt("random_weights.txt", weights, delimiter="\t")
weights = np.loadtxt("random_weights.txt", delimiter="\t")
weights

array([0.1906199 , 0.09999955, 0.55694646, 0.35209264, 0.62699634,
       0.73718176, 0.32825287, 0.21433344, 0.15181921, 0.88893431,
       0.81016578, 0.04990381, 0.52550728, 0.69759923, 0.62925021,
       0.62941622, 0.57071457, 0.25755489, 0.83517121, 0.25747145,
       0.19257471, 0.23877505, 0.46084473, 0.54087286, 0.89903685,
       0.38446579, 0.77849082, 0.77669476, 0.55122692, 0.65886365,
       0.82031978, 0.29321274, 0.58652881, 0.28261414, 0.57706136,
       0.76777787, 0.67435458, 0.40959194, 0.69497444, 0.40516409,
       0.67729062, 0.99349098, 0.95791314, 0.65888598, 0.00654588,
       0.35789028, 0.59185539, 0.97765869, 0.87035596, 0.9267479 ])

In [24]:
result = list()
for x in windows:
    result.append(ls.eigen_windows(x, 10, 1))
result = np.vstack(result)
pc_dists = ls.get_pc_dists(result)
mds = pcoa(pc_dists)
mds_likelostructr = mds


/home/josephguhlin/anaconda3/envs/gwasp/lib/python3.8/site-packages/numpy/core/_asarray.py:85: ComplexWarning:

Casting complex values to real discards the imaginary part

/mnt/g/Development/lostruct-py/lostructpy/lostruct.py:105: ComplexWarning:

Casting complex values to real discards the imaginary part



In [25]:
mds_coords = pd.read_csv("lostruct-results/mds_coords.csv")
np.corrcoef(mds.samples['PC1'], mds_coords['MDS1'].to_numpy())

array([[1.       , 0.9978494],
       [0.9978494, 1.       ]])

In [26]:
px.scatter(x=mds.samples["PC1"], y=mds_coords['MDS1'])

In [8]:
px.scatter(y=mds.samples["PC1"])

In [9]:
px.scatter(y=mds.samples["PC2"])

## Comparison with Weights

In [33]:
result = list()
for x in windows:
    result.append(ls.eigen_windows(x, 10, weights))
result = np.vstack(result)
pc_dists = ls.get_pc_dists(result)
mds = pcoa(pc_dists)
mds_likelostructr = mds
mds_coords = pd.read_csv("lostruct-results/mds_coords.csv")
print("Weights compared to unweighted Lostruct R:")
print(np.corrcoef(mds.samples['PC1'], mds_coords['MDS1'].to_numpy()))

Weights compared to unweighted Lostruct R:
[[1.         0.84853512]
 [0.84853512 1.        ]]


In [31]:
px.scatter(x=mds.samples["PC1"], y=mds_coords['MDS1'])

# Some looks at other methods of clustering / comparing

In [10]:
embedding = MDS(n_components=10, dissimilarity="precomputed", n_jobs=-1, n_init=32)
mds = embedding.fit_transform(pc_dists)
px.scatter(y=[mds[:,0], mds_coords['MDS1']])

In [11]:
import phate
phater = phate.PHATE(n_components=10, knn_dist='precomputed', mds_solver='smacof', mds='metric')
comparison_phate = phater.fit_transform(pc_dists)

Calculating PHATE...
  Running PHATE on precomputed distance matrix with 124 observations.
  Calculating graph and diffusion operator...
    Calculating affinities...
  Calculating optimal t...
    Automatically selected t = 14
  Calculated optimal t in 0.01 seconds.
  Calculating diffusion potential...
  Calculating metric MDS...
  Calculated metric MDS in 0.22 seconds.
Calculated PHATE in 0.25 seconds.


In [12]:
px.scatter(y=[mds_coords['MDS1'], mds_likelostructr.samples["PC1"], comparison_phate[:,0]])

In [13]:
reducer = umap.UMAP()
embedding = reducer.fit_transform(pc_dists)
px.scatter(x=embedding[:, 0], y=embedding[:, 1])

In [14]:
hdbscan_labels = hdbscan.HDBSCAN().fit_predict(embedding)
px.scatter(x=embedding[:, 0], y=embedding[:, 1], color=hdbscan_labels)

In [15]:
reducer = umap.UMAP(n_components=3)
embedding = reducer.fit_transform(pc_dists)
hdbscan_labels = hdbscan.HDBSCAN().fit_predict(embedding)
fig = px.scatter_3d(x=embedding[:, 0], y=embedding[:, 1], z=embedding[:, 2], color=hdbscan_labels, width=800, height=600)
fig.show()

In [16]:
result[0][0]

masked_array(
  data=[[ 2.02479338e-03, -1.56257819e-04,  3.15643633e-04, ...,
         -1.95499743e-04, -1.99238267e-03,  5.25224767e-04],
        [-2.97859829e-04,  3.93349248e-03, -3.39655643e-04, ...,
         -2.23627533e-03, -5.61482114e-03, -5.65180276e-04],
        [ 1.08031851e-04, -6.09850580e-05,  5.77912055e-04, ...,
          5.81543681e-04, -6.75933226e-04,  9.61634236e-04],
        ...,
        [-3.81177414e-05, -2.28736812e-04,  3.31290152e-04, ...,
          4.50960872e-02, -8.02952672e-04,  5.51260264e-04],
        [-4.36359142e-04, -6.45114898e-04, -4.32534082e-04, ...,
         -9.01945511e-04,  9.72008320e-02, -7.19728162e-04],
        [ 1.08031851e-04, -6.09850580e-05,  5.77912055e-04, ...,
          5.81543681e-04, -6.75933226e-04,  9.61634236e-04]],
  mask=False,
  fill_value=1e+20)

In [17]:
testmat = np.matrix([[1, 4, 7], [2,5,8], [3,6,9]])
testmat

matrix([[1, 4, 7],
        [2, 5, 8],
        [3, 6, 9]])

In [18]:
sqrt_w = np.sqrt(np.array([1,2,4]))[np.newaxis]
sqrt_w

array([[1.        , 1.41421356, 2.        ]])

In [19]:
sqrt_w * testmat

matrix([[ 9.82842712, 23.07106781, 36.3137085 ]])

In [20]:
sqrt_w[np.newaxis].T

array([[[1.        ]],

       [[1.41421356]],

       [[2.        ]]])

In [21]:
testmat * np.tile(sqrt_w, (3, 1))

matrix([[12.        , 16.97056275, 24.        ],
        [15.        , 21.21320344, 30.        ],
        [18.        , 25.45584412, 36.        ]])

In [22]:
x = np.multiply(testmat, sqrt_w.T)
x

matrix([[ 1.        ,  4.        ,  7.        ],
        [ 2.82842712,  7.07106781, 11.3137085 ],
        [ 6.        , 12.        , 18.        ]])

In [23]:
np.multiply(x, sqrt_w)

matrix([[ 1.        ,  5.65685425, 14.        ],
        [ 2.82842712, 10.        , 22.627417  ],
        [ 6.        , 16.97056275, 36.        ]])